Playing around with odometry. We won't have numpy on the Yukon but will prototype with it, then reproduce in pure python to put on the mcu.

In [6]:
import numpy as np


WHEEL_RADIUS = 0.030
TRACK_WIDTH  = 0.195 
TRACK_LENGTH = 0.150

l = TRACK_LENGTH / 2
w = TRACK_WIDTH / 2
r = WHEEL_RADIUS

fwk_front_left  = [-l-w, 1,  1]
fwk_front_right = [ l+w, 1, -1]
fwk_rear_left   = [-l-w, 1, -1]
fwk_rear_right  = [ l+w, 1,  1]

ik_const = (l+w)

# ik_front_left  = -ik_const
# ik_front_right =  ik_const
# ik_rear_left   = -ik_const
# ik_rear_right  =  ik_const

kinematic_model = np.array([fwk_front_left, 
                            fwk_front_right,
                            fwk_rear_left,
                            fwk_rear_right])

forward_kinematic_model = np.multiply(kinematic_model, 1/r)

odom_model = np.array([[ 1,  1,  1,  1],
                       [ 1, -1, -1, 1],
                       [-ik_const,  ik_const,  -ik_const, ik_const]])


inverse_kinematic_model = np.multiply(odom_model, r/4) 

def calc_motor_speeds(vx, vy, wz):
    input_array = np.array([wz,vx,vy])    
    input_array.reshape(3, 1)
    print(input_array)
    print(forward_kinematic_model)
    u = np.multiply(forward_kinematic_model, input_array)
    return u

forward_wheel_speeds = calc_motor_speeds(1, 0, 0)
backward_wheel_speeds = calc_motor_speeds(-1, 0, 0)
left_wheel_speeds = calc_motor_speeds(0, -1, 0)
right_wheel_speeds = calc_motor_speeds(0, 1, 0)
left_rotate_wheel_speeds = calc_motor_speeds(0, 0, 1)
right_rotate_wheel_speeds = calc_motor_speeds(0, 0, -1)
random_wheel_speeds = calc_motor_speeds(1.76, -0.67, 3.52)

print(forward_wheel_speeds)

[0 1 0]
[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75        33.33333333 -33.33333333]
 [  5.75        33.33333333  33.33333333]]
[ 0 -1  0]
[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75        33.33333333 -33.33333333]
 [  5.75        33.33333333  33.33333333]]
[ 0  0 -1]
[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75        33.33333333 -33.33333333]
 [  5.75        33.33333333  33.33333333]]
[0 0 1]
[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75        33.33333333 -33.33333333]
 [  5.75        33.33333333  33.33333333]]
[1 0 0]
[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75        33.33333333 -33.33333333]
 [  5.75        33.33333333  33.33333333]]
[-1  0  0]
[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75      

For the inverse kinematic model, is it just reshaping the array and dividing the bottom row by 1?

In [7]:
print(f"Forward Kinematic Array:\n {kinematic_model}")
print(f"Odom Model Array:\n {odom_model}")

Forward Kinematic Array:
 [[-0.1725  1.      1.    ]
 [ 0.1725  1.     -1.    ]
 [-0.1725  1.     -1.    ]
 [ 0.1725  1.      1.    ]]
Odom Model Array:
 [[ 1.      1.      1.      1.    ]
 [ 1.     -1.     -1.      1.    ]
 [-0.1725  0.1725 -0.1725  0.1725]]


No, but it looks like it may be before they're adjusted by /r or r/4...

In [8]:
print(f"Kinematic Array:\n {kinematic_model}")
print(f"Odom Model Array:\n {odom_model}")

Kinematic Array:
 [[-0.1725  1.      1.    ]
 [ 0.1725  1.     -1.    ]
 [-0.1725  1.     -1.    ]
 [ 0.1725  1.      1.    ]]
Odom Model Array:
 [[ 1.      1.      1.      1.    ]
 [ 1.     -1.     -1.      1.    ]
 [-0.1725  0.1725 -0.1725  0.1725]]


In [9]:
forward_velocities = np.dot(odom_model, forward_wheel_speeds)
backward_velocities = np.dot(odom_model, backward_wheel_speeds)
left_velocities = np.dot(odom_model, left_wheel_speeds)
right_velocities = np.dot(odom_model, right_wheel_speeds)
left_rotate_velocities = np.dot(odom_model, left_rotate_wheel_speeds)
right_rotate_velocities = np.dot(odom_model, right_rotate_wheel_speeds)
random_velocities = np.dot(odom_model, random_wheel_speeds)
print(np.round(forward_velocities))
print(np.round(backward_velocities))
print(np.round(left_velocities, 4))
print(np.round(right_velocities, 4))
print(np.round(left_rotate_velocities, 4))
print(np.round(right_rotate_velocities, 4))
print(np.round(random_velocities, 4))

[[  0. 133.   0.]
 [  0.   0.   0.]
 [  0.  -0.   0.]]
[[   0. -133.    0.]
 [   0.    0.    0.]
 [   0.    0.    0.]]
[[   0.        0.        0.    ]
 [   0.        0.     -133.3333]
 [   0.        0.        0.    ]]
[[  0.       0.       0.    ]
 [  0.       0.     133.3333]
 [  0.       0.      -0.    ]]
[[0.     0.     0.    ]
 [0.     0.     0.    ]
 [3.9675 0.     0.    ]]
[[ 0.      0.      0.    ]
 [ 0.      0.      0.    ]
 [-3.9675  0.      0.    ]]
[[  0.     234.6667   0.    ]
 [  0.       0.     -89.3333]
 [ 13.9656   0.      -0.    ]]


In [10]:
import array 
import mtx
import time

def matrix_multiply(a, b):
    result = [[0 for _ in range(len(b[0]))] for _ in range(len(a))]

    for i in range(len(a)):
        for j in range(len(b[0])):
            for k in range(len(b)):
                result[i][j] += a[i][k] * b[k][j]

    return result

odom_model_nnp = [array.array('f', [ 0.0075,      0.0075,      0.0075,      0.0075]),
                  array.array('f', [ 0.0075,     -0.0075,     -0.0075,      0.0075]),
                  array.array('f', [-0.04347826,  0.04347826, -0.04347826,  0.04347826])]

print(f"IKM: {inverse_kinematic_model}")
print(f"RWS: {random_wheel_speeds}")
dot_time = time.time_ns()
random_dot_product = np.multiply(inverse_kinematic_model, random_wheel_speeds)
dot_time = time.time_ns() - dot_time

mtx_time = time.time_ns()
mtx_mulled = mtx.mul(inverse_kinematic_model, random_wheel_speeds)
mtx_time = time.time_ns() - mtx_time

#print(f"{random_velocities})
print(f"multiply: {random_dot_product}")
print(f"mtx: {mtx_mulled}")
print(f"dot time: {dot_time/1000000}ms, mtx time: {mtx_time/1000000}ms")
#print(odom_model)

IKM: [[ 0.0075      0.0075      0.0075      0.0075    ]
 [ 0.0075     -0.0075     -0.0075      0.0075    ]
 [-0.00129375  0.00129375 -0.00129375  0.00129375]]
RWS: [[-20.24        58.66666667 -22.33333333]
 [ 20.24        58.66666667  22.33333333]
 [-20.24        58.66666667  22.33333333]
 [ 20.24        58.66666667 -22.33333333]]


ValueError: operands could not be broadcast together with shapes (3,4) (4,3) 

MTX is twice as slow so we'll go with the simple function. To calulate distance and rotation from a start point, we need to take the speed of the wheels with a duration and multiply it by the odom model to get the velocities and angular momentum.


In [11]:
# Take motor speeds and a duration, return the distance traveled and change in rotation
# SI Units used so meters and radians will be returned
def calc_distances(motor_speeds, duration):
    velocities = matrix_multiply(odom_model, motor_speeds)
    return velocities[0][0] * duration, velocities[1][0], velocities[2][0] * duration

So, if we want to go forward for one second at 1m/s:

In [12]:
forward_motor_speeds = calc_motor_speeds(1, 0, 0)
print(f"Motor Speeds: {forward_motor_speeds}")

odom = calc_distances(forward_motor_speeds, 2)
print(f"Odom: {odom}")

[0 1 0]
[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75        33.33333333 -33.33333333]
 [  5.75        33.33333333  33.33333333]]
Motor Speeds: [[-0.         33.33333333  0.        ]
 [ 0.         33.33333333 -0.        ]
 [-0.         33.33333333 -0.        ]
 [ 0.         33.33333333  0.        ]]
Odom: (0.0, 0.0, 0.0)


So, to invert the forward kinematic array we need to create a new one and invert a few values. 

Where for the forward kinematic we have:

```
-l-w,   1,  1
 l+w,   1, -1
-l-w,   1, -1
 l+w,   1,  1
```

We need to reshape it and calculate the reciprical of a few values
```
1,          -1,         -1,          1
1,           1,          1,          1
1/(-l-w)     1/(l+w)     1/(-l-w)    1/(l+w)
```

In [ ]:
# This works on a shallow copy of A so the original is modified
def matrix_scalar(a, b):
    for i in range(len(a)):
        for j in range(len(a[0])):
               a[i][j] = a[i][j] * b


WHEEL_RADIUS = 0.030
TRACK_WIDTH  = 0.195 
TRACK_LENGTH = 0.150

l = TRACK_LENGTH / 2
w = TRACK_WIDTH / 2
r = WHEEL_RADIUS

fwk_front_left  = array.array('f', [-l-w, 1,  1])
fwk_front_right = array.array('f', [ l+w, 1, -1])
fwk_rear_left   = array.array('f', [-l-w, 1, -1])
fwk_rear_right  = array.array('f', [ l+w, 1,  1])

fkm = [fwk_front_left, 
       fwk_front_right,
       fwk_rear_left,
       fwk_rear_right] 

ikm = [array.array('f', [  fkm[0][1],   fkm[1][1],   fkm[2][1],   fkm[3][1]]),
       array.array('f', [  fkm[0][2],   fkm[1][2],   fkm[2][2],   fkm[3][2]]),
       array.array('f', [1/fkm[0][0], 1/fkm[1][0], 1/fkm[2][0], 1/fkm[3][0]])]

matrix_scalar(fkm, 1/r)
matrix_scalar(ikm, r/4)

print(forward_kinematic_model)
print(fkm)

print(inverse_kinematic_model)
print(ikm)



[[ -5.75        33.33333333  33.33333333]
 [  5.75        33.33333333 -33.33333333]
 [ -5.75        33.33333333 -33.33333333]
 [  5.75        33.33333333  33.33333333]]
[array('f', [-5.75, 33.33333206176758, 33.33333206176758]), array('f', [5.75, 33.33333206176758, -33.33333206176758]), array('f', [-5.75, 33.33333206176758, -33.33333206176758]), array('f', [5.75, 33.33333206176758, 33.33333206176758])]
[[ 0.0075      0.0075      0.0075      0.0075    ]
 [ 0.0075     -0.0075     -0.0075      0.0075    ]
 [-0.00129375  0.00129375 -0.00129375  0.00129375]]
[array('f', [0.007499999832361937, 0.007499999832361937, 0.007499999832361937, 0.007499999832361937]), array('f', [0.007499999832361937, -0.007499999832361937, -0.007499999832361937, 0.007499999832361937]), array('f', [-0.043478261679410934, 0.043478261679410934, -0.043478261679410934, 0.043478261679410934])]


```

-0.172, 1.0, -1.0
 0.172, 2.0, -2.0
-0.172, 3.0, -3.0
 0.172, 4.0, -4.0

 1.0, 2.0, 3.0, 4.0
-1.0, -2.0, -3.0, -4.0
-0.172, 0.172, -0.172, 0.172

```

In [ ]:
import numpy as np

print(forward_kinematic_model)

def calc_motor_speeds(vx, vy, wz):
    input_array = np.array([wz,vx,vy])    
    input_array.shape = (3, 1)
    u = np.multiply(forward_kinematic_model, input_array)
    return u

speeds = calc_motor_speeds(0, 1, 0)
print(speeds)


NameError: name 'forward_kinematic_model' is not defined